## Import Packages

In [1]:
from BaseBlocks import *
from SimulinkBlockClass import SimulinkBlock
from SimulationClass import Simulation

## Build the model 

In [20]:
block = SimulinkBlock('test',['in1','in2'],['out1'])

block.addBlock(Add(name='add',pos1='in1',pos2='param2',output='added'))
block.addCalibrationParameter(Parameter(name='param2'))

block.addConnection(name='in1',data='in1',start=block.blocks['Input'],end=block.blocks['add'])
block.addConnection(name='param2',data='param2',start=block.blocks['param2'],end=block.blocks['add'])

block.addBlock(Subtract(name='sub',pos='in2',neg='added',output='subtracted'))
block.addConnection(name='in2',data='in2',start=block.blocks['Input'],end=block.blocks['sub'])
block.addConnection(name='added',data='added',start=block.blocks['add'],end=block.blocks['sub'])

block.addCalibrationParameter(Parameter(name='param1'))
block.addBlock(Multiply(name='mul',factor1='param1',factor2='subtracted',output='out1'))

block.addConnection(name='sub-mul',data='subtracted',start=block.blocks['sub'],end=block.blocks['mul'])
block.addConnection(name='param1-mul',data='param1',start=block.blocks['param1'],end=block.blocks['mul'])

block.addConnection(name='out1',data='out1',start=block.blocks['mul'],end=block.blocks['Output'])

block.compileBlock()

Block compiled successfully!


In [14]:
block.getCalibrationInfo()

Calibration parameters for test:
    param2 is not calibrated
    param1 is not calibrated


## Visualize the model

In [21]:
block.visualizeModel()

## Run the simulation

In [97]:
import pandas as pd

data = pd.DataFrame({'in1':np.arange(10_000),'in2':np.arange(10_000)})

simulation = Simulation(block,data)

simulation.getCalibrationParameters()
simulation.setCalibration(param1=[1],param2=[1])

simulation.runSimulation()

Calibratable parameters: param2 param1
param2 calibrated to [0 1 2 3 4]
param1 calibrated to [0 1 2 3 4]


In [98]:
import polars as pl

df = pl.scan_parquet('outputs/test/out1/output.parquet').max().collect()
#t = df.to_numpy().reshape((101,50))
#
#import seaborn as sns
#
#sns.heatmap(t)